Next steps:
1. Add a "master node" that connects all nodes together, so that message passing works between all nodes

In [56]:
from LigandBinaryDataset import LigandBinaryDataset

import yaml
import torch
from torch_geometric.loader import DataLoader

with open('hyperparameters.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [57]:
dataset = LigandBinaryDataset('./data/')

In [58]:
dataset = dataset.shuffle()
train_dataset = dataset[:int(len(dataset) * 0.7)]
val_dataset = dataset[int(len(dataset) * 0.7):int(len(dataset) * 0.85)]
test_dataset = dataset[int(len(dataset) * 0.85):]

In [59]:
train_dl = DataLoader(train_dataset, batch_size=config['BATCH_SIZE'])
val_dl = DataLoader(val_dataset, batch_size=config['BATCH_SIZE'])
test_dl = DataLoader(test_dataset, batch_size=config['BATCH_SIZE'])

In [85]:
from LigandGNNV1 import LigandGNNV1

device = torch.device('cuda')
model = LigandGNNV1(dataset.num_node_features, dataset.num_classes).to(device)

In [93]:
model(dataset[0].to(device))

tensor([[ 14.0451,  -9.9288],
        [ 18.7868,  -7.3266],
        [ 16.4257,   2.8898],
        ...,
        [ 61.5646, -49.6569],
        [ 61.2142, -36.6363],
        [ 59.1271, -30.5633]], device='cuda:0', grad_fn=<AddBackward0>)

In [91]:
len(dataset[0].y)

600